# Guide pour l'Appel à une API Hugging Face pour la Segmentation d'Images

Bienvenue ! Ce notebook a pour but de vous guider pas à pas dans l'utilisation de l'API d'inférence de Hugging Face pour effectuer de la segmentation d'images. La segmentation d'images consiste à attribuer une étiquette (comme "cheveux", "vêtement", "arrière-plan") à chaque pixel d'une image.

Nous allons :
1. Comprendre ce qu'est une API et comment s'y connecter.
2. Envoyer une image à un modèle de segmentation hébergé sur Hugging Face.
3. Récupérer et interpréter les résultats.
4. Visualiser les masques de segmentation.
5. Étendre cela pour traiter plusieurs images.

### 1. Configuration Initiale et Importations

Nous commençons par importer toutes les bibliothèques Python nécessaires pour le projet de segmentation d’images.

<small>

```python
# --- Imports standards ---
import os           # interagir avec le système de fichiers (lister les images, créer des dossiers, etc.)
import time         # mesurer les temps de traitement
import asyncio      # gérer les appels asynchrones (utile pour plusieurs images en parallèle)
import requests     # effectuer des requêtes HTTP vers des API

# --- Imports tiers ---
import numpy as np                             # manipulation des tableaux numériques (images comme tableaux de pixels)
import torch                                    # tenseurs et modèles deep learning
from PIL import Image                           # lecture, écriture et manipulation des images
from tqdm.notebook import tqdm                  # barre de progression interactive dans Jupyter Notebook
import aiohttp                                   # requêtes HTTP asynchrones
from transformers import AutoModelForSemanticSegmentation, AutoImageProcessor
                                               # modèles pré-entraînés pour la segmentation d’images
from dotenv import load_dotenv                  # charger des variables d'environnement depuis un fichier .env
import matplotlib.pyplot as plt                 # affichage des images et des masques de segmentation


In [ ]:
# --- Imports standards ---
import os
import time
import asyncio
import requests

# --- Imports tiers ---
import numpy as np
import torch
from PIL import Image
from tqdm.notebook import tqdm
import aiohttp
from transformers import AutoModelForSemanticSegmentation, AutoImageProcessor
from dotenv import load_dotenv


# Pause (seconds) entre appels API pour être courtois et éviter le rate limiting
PAUSE_BETWEEN_CALLS = 1.5

### Variables de Configuration

Nous devons définir quelques variables :
- `images_dir`: Le chemin vers le dossier contenant vos images. **Assurez-vous de modifier ce chemin si nécessaire.**
- `predicted_masks_dir`: Le chemin vers le dossier contenant les masques prédits. **Assurez-vous de modifier ce chemin si nécessaire.**
- `ground_truth_masks_dir`: Le chemin vers le dossier où les masques de vérité terrain sont sauvegardés. **Assurez-vous de modifier ce chemin si nécessaire.**

⚠️ **IMPORTANT :** Merci d'utiliser le même nommage pour vos images et vos masques (image_01 -> mask_01)

- `max_images`: Le nombre maximum d'images à traiter (pour ne pas surcharger l'API ou attendre trop longtemps).
- `api_token`: Votre jeton d'API Hugging Face. **IMPORTANT : Gardez ce jeton secret !**

**Comment obtenir un token API Hugging Face ?**
1. Créez un compte sur [huggingface.co](https://huggingface.co/).
2. Allez dans votre profil -> Settings -> Access Tokens.
3. Créez un nouveau token (par exemple, avec le rôle "read").
4. Copiez ce token ici.

In [ ]:
# Configuration pour le projet Fashion Trend Intelligence
# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

# Utiliser la variable d'environnement pour le token API (plus sécurisé)
api_token = os.getenv("HUGGINGFACE_API_KEY")

# Mise à jour des chemins pour la structure de votre projet
images_dir = "../assets/images"  # Images stockées dans le dossier assets
predicted_masks_dir = "../assets/predicted_masks"  # Masques de vérité terrain
ground_truth_masks_dir = "../assets/ground_truth_masks"  # Masques de vérité terrain
max_images = 1  # Commençons avec peu d'images

# Créer le dossier assets s'il n'existe pas
if not os.path.exists(images_dir):
    os.makedirs(images_dir)
    print(f"✅ Dossier images '{images_dir}' créé. Veuillez y ajouter des images.")
else:
    print(f"✅ Dossier images '{images_dir}' existant.")

if not os.path.exists(predicted_masks_dir):
    os.makedirs(predicted_masks_dir)
    print(f"✅ Dossier masques '{predicted_masks_dir}' créé. Veuillez y ajouter des masques.")
else:
    print(f"✅ Dossier masques '{predicted_masks_dir}' existant.")

if not os.path.exists(ground_truth_masks_dir):
    os.makedirs(ground_truth_masks_dir)
    print(f"✅ Dossier masques de vérité terrain '{ground_truth_masks_dir}' créé. Veuillez y ajouter des masques.")
else:
    print(f"✅ Dossier masques de vérité terrain '{ground_truth_masks_dir}' existant.")

if not api_token or api_token == "your_huggingface_api_key_here":
    print("\n⚠️  ATTENTION : Veuillez configurer votre HUGGINGFACE_API_KEY dans le fichier .env")
    print("📖 Obtenez votre token depuis : https://huggingface.co/settings/tokens")
else:
    print(f"✅ Token API chargé avec succès (se terminant par : ...{api_token[-8:]})")

## 2. Comprendre l'API d'Inférence Hugging Face

L'API d'inférence permet d'utiliser des modèles hébergés sur Hugging Face sans avoir à les télécharger ou à gérer l'infrastructure.

- **Modèle utilisé** : Nous allons utiliser le modèle `sayeed99/segformer_b3_clothes`, spécialisé dans la segmentation de vêtements et de parties du corps.
- **URL de l'API** : L'URL pour un modèle est généralement `https://api-inference.huggingface.co/models/NOM_DU_MODELE`.
- **Headers (En-têtes)** : Pour s'authentifier et spécifier le type de contenu, nous envoyons des en-têtes avec notre requête.
    - `Authorization`: Contient notre token API (précédé de `Bearer `).
    - `Content-Type`: Indique que nous envoyons une image au format JPEG (ou PNG selon le cas).

In [ ]:
%load_ext autoreload
%autoreload 2

API_URL = "https://router.huggingface.co/hf-inference/models/sayeed99/segformer_b3_clothes"  # Remplacez ... par le bon endpoint.
headers = {
    "Authorization": f"Bearer {api_token}"
    # Le "Content-Type" sera ajouté dynamiquement lors de l'envoi de l'image
}

# Import des modules utilitaires pour les images et les connexions
import oc_p2_fashion_trend_intelligence.utils.image_utils as image_utils
import oc_p2_fashion_trend_intelligence.utils.connection_utils as connection_utils


## 3. Configuration et Validation des Images

Cette section regroupe **toutes les constantes configurables** du notebook pour faciliter la personnalisation :

**📝 Configuration modifiable :**
- `ALLOWED_EXTENSIONS`: Extensions d'images supportées 
- `CLASS_MAPPING`: Classes spécifiques au modèle `segformer_b3_clothes`
- `PALETTE`: Couleurs pour visualiser les masques de segmentation

**🔍 Validation automatique :**
- Listing des images dans le dossier défini
- Validation du format et de la taille des images


💡 **Astuce** : Modifiez `ALLOWED_EXTENSIONS` si vous voulez supporter d'autres formats d'image !

In [ ]:
# =============================================================================
# CONFIGURATION DU NOTEBOOK - MODIFIEZ SELON VOS BESOINS
# =============================================================================

# Extensions d'images supportées
ALLOWED_EXTENSIONS = (".png", ".jpg", ".jpeg", ".bmp", ".webp")

# Configuration spécifique au modèle segformer_b3_clothes
CLASS_MAPPING = {
    "Background": 0,
    "Hat": 1,
    "Hair": 2,
    "Sunglasses": 3,
    "Upper-clothes": 4,
    "Skirt": 5,
    "Pants": 6,
    "Dress": 7,
    "Belt": 8,
    "Left-shoe": 9,
    "Right-shoe": 10,
    "Face": 11,
    "Left-leg": 12,
    "Right-leg": 13,
    "Left-arm": 14,
    "Right-arm": 15,
    "Bag": 16,
    "Scarf": 17
}

# Palette de couleurs
# Les couleurs proches pour bras, jambes et chaussures permettent de reconnaître la symétrie.
# Les accessoires (Hat, Bag, Scarf) ont des teintes distinctes pour les différencier facilement.
# Fond noir (Background) pour faire ressortir les vêtements.
# Les couleurs vives aident à visualiser clairement les masques sur l’image.

PALETTE = {
    0:  [0, 0, 0],    # Background = noir
    1:  [255, 128, 0],      # Hat = orange
    2:  [255, 192, 203],    # Hair = rose clair
    3:  [0, 255, 255],      # Sunglasses = cyan
    4:  [255, 0, 0],        # Upper-clothes = rouge
    5:  [255, 165, 0],      # Skirt = orange clair
    6:  [0, 128, 0],        # Pants = vert foncé
    7:  [128, 0, 128],      # Dress = violet
    8:  [128, 64, 0],       # Belt = marron
    9:  [0, 0, 255],        # Left-shoe = bleu
    10: [0, 0, 200],        # Right-shoe = bleu plus foncé
    11: [255, 224, 189],    # Face = couleur peau claire
    12: [0, 255, 0],        # Left-leg = vert clair
    13: [0, 200, 0],        # Right-leg = vert foncé
    14: [255, 255, 0],      # Left-arm = jaune
    15: [200, 200, 0],      # Right-arm = jaune foncé
    16: [128, 0, 0],        # Bag = rouge foncé
    17: [0, 128, 128]       # Scarf = teal / bleu-vert
}


# =============================================================================
# LISTING ET VALIDATION DES IMAGES
# =============================================================================

# Lister les chemins des images à traiter
image_paths = [
    os.path.join(images_dir, filename) 
    for filename in os.listdir(images_dir) 
    if filename.endswith(ALLOWED_EXTENSIONS)
]
# Tri des images par nom du fichier, identifiant le _xx après image_
# pour traiter les images dans l'ordre (image_01, image_02, ..., image_10, image_11, ...)
# et non pas (image_1, image_10, image_11, image_2, ...)
image_paths.sort(
    key=lambda x: os.path.basename(x).split("_")[1],
    reverse=True
)

# Valider les images (format, taille, etc.)
valid_images = [img_path for img_path in image_paths if image_utils.validate_single_image(img_path, max_size=(1024, 1024))]

if not valid_images:
    print(f"Aucune image valide trouvée dans '{images_dir}'. Veuillez y ajouter des images.")
else:
    print(f"{len(valid_images)} image(s) valide(s) sur {len(image_paths)} à traiter.")
    
# Les fonctions utilitaires (get_image_dimensions, decode_base64_mask, create_masks) 
# sont maintenant disponibles via l'import depuis image_utils

## 4. Segmentation d'une Seule Image

Avant de traiter toutes les images, concentrons-nous sur une seule pour bien comprendre le processus.

⚠️ Le service utilisé sayeed99/segformer_b3_clothes ne semble pas stable et peut régulièrement entrâiner des timeouts. Si vous rencontrez des erreurs, RDV à l'Annexe 3   pour une version locale.

Étapes :
1.  Choisir une image.
2.  Ouvrir l'image en mode binaire (`"rb"`) et lire son contenu (`data`).
3.  Déterminer le `Content-Type` (par exemple, `"image/jpeg"` ou `"image/png"`).
4.  Envoyer la requête POST à l'API avec `requests.post()` en passant l'URL, les headers et les données.
5.  Vérifier le code de statut de la réponse. Une erreur sera levée si le code n'est pas 2xx (succès) grâce à `response.raise_for_status()`.
6.  Convertir la réponse JSON en un dictionnaire Python avec `response.json()`.
7.  Utiliser nos fonctions `get_image_dimensions` et `create_masks` pour obtenir le masque final.
8.  Sauvegarder le masque dans le dossier défini.
9.  Afficher l'image originale et le masque segmenté.

In [ ]:
if image_paths:
    single_image_path = image_paths[3]
    print(f"Traitement de l'image : {single_image_path}")

    # Lecture du fichier
    try:
        with open(single_image_path, "rb") as image_file:
            image_data = image_file.read()
    except OSError as e:
        print(f"📂 Erreur lecture fichier : {e}")
        raise RuntimeError("Arrêt de la cellule car fichier non lisible")

    # Déterminer dynamiquement le content-type
    ext = os.path.splitext(single_image_path)[1].lower()
    headers["Content-Type"] = f"image/{ext[1:]}"

    # Appel API
    response = connection_utils.post_with_retries(API_URL, headers=headers, data=image_data)
    if response is None:
        raise RuntimeError("❌ Impossible de segmenter l'image → arrêt de la cellule")

    print("✅ Segmentation réussie !")

    # Traitement du résultat
    try:
        segmentation_result = response.json()
        width, height = image_utils.get_image_dimensions(single_image_path)
        mask = image_utils.create_masks(segmentation_result, width, height, CLASS_MAPPING)
    except Exception as e:
        print(f"⚠️ Erreur lors du traitement de la segmentation : {e}")
        raise RuntimeError("Arrêt de la cellule suite à une erreur dans le masque")

    # Sauvegarde du masque
    mask_save_path = os.path.join(
        predicted_masks_dir, os.path.basename(single_image_path).replace('image', 'mask')
    )
    Image.fromarray(mask).save(mask_save_path)
    print(f"🖼️ Masque sauvegardé : {mask_save_path}")

    # Affichage
    image_utils.show_image_and_masks(
        single_image_path,
        ground_truth_masks_dir,
        CLASS_MAPPING,
        PALETTE,
        predicted_masks_dir, None
    )

    # Evaluation simple si masque de vérité terrain disponible
    ground_truth_mask_path = image_utils.search_for_ground_truth_mask_path(
        single_image_path, ground_truth_masks_dir
    )
    if ground_truth_mask_path:
        print(image_utils.eval_segmentation_simple(ground_truth_mask_path, mask_save_path, CLASS_MAPPING, PALETTE))

else:
    print(
        "Aucune image à traiter. Vérifiez la configuration de 'images_dir' et 'max_images'."
    )

## 5. Segmentation de Plusieurs Images (Batch)

Maintenant que nous savons comment traiter une image, nous pouvons créer une fonction pour en traiter plusieurs.
Cette fonction va boucler sur la liste `image_paths` et appliquer la logique de segmentation à chaque image.
Nous utiliserons `tqdm` pour avoir une barre de progression.

In [ ]:
def segment_images_batch(list_of_image_paths):
    """
    Segmente une liste d'images en utilisant l'API Hugging Face.

    Args:
        list_of_image_paths (list): Liste des chemins vers les images.

    Returns:
        list: Liste des masques de segmentation (tableaux NumPy).
              Contient None si une image n'a pas pu être traitée.
    """
    batch_segmentations = []

    # N'oubliez pas de mettre une pause entre chaque appel API !
    for img_path in tqdm(list_of_image_paths, desc="Segmentation des images en batch"):
        mask = None
        try:
            # Lecture fichier image
            with open(img_path, "rb") as image_file:
                image_data = image_file.read()
        except OSError as e:
            print(f"📂 Erreur lecture fichier {img_path} : {e}")
            batch_segmentations.append(None)
            continue  # passe directement à l’image suivante

        headers["Content-Type"] = "image/png"
        response = connection_utils.post_with_retries(API_URL, headers=headers, data=image_data)

        if response is not None:
            try:
                segmentation_result = response.json()
                width, height = image_utils.get_image_dimensions(img_path)
                mask = image_utils.create_masks(segmentation_result, width, height, CLASS_MAPPING)
                # Sauvegarde du masque
                mask_save_path = os.path.join(
                    predicted_masks_dir, os.path.basename(img_path).replace('image', 'mask')
                )
                Image.fromarray(mask).save(mask_save_path)
                print(f"🖼️ Masque sauvegardé : {mask_save_path}")
            except Exception as e:
                print(f"⚠️ Erreur lors du traitement de {img_path} : {e}")
                mask = None
        else:
            print(f"❌ API n'a pas renvoyé de résultat pour {img_path}")

        batch_segmentations.append(mask)

        # Pause entre les appels API
        time.sleep(PAUSE_BETWEEN_CALLS)

    return batch_segmentations

# Appeler la fonction pour segmenter les images listées dans image_paths
if image_paths:
    print(f"\nTraitement de {len(image_paths)} image(s) en batch...")
    batch_seg_results = segment_images_batch(image_paths)
    print("Traitement en batch terminé.")
else:
    batch_seg_results = []
    print("Aucune image à traiter en batch.")

## 6. Affichage des Résultats

Nous allons maintenant créer une fonction pour afficher le résultat des traitements : 
- les images originales,
- leur masque prédit,
- le masque de vérité terrain,
- des métriques d'évaluation (IoU, Accuracy).


In [ ]:
def display_results(original_image_paths):
    """
    Affiche les images originales et leurs masques segmentés à partir des chemins fournis.

    Args:
        original_image_paths (list): Liste des chemins des images originales.
    """
    num_images = len(original_image_paths)
    print(f"\nAffichage des résultats des {num_images} images...")

    image_with_best_accuracy = None
    best_accuracy = 0.0
    image_with_worst_accuracy = None  # --- IGNORE ---
    worst_accuracy = 1.0

    image_with_best_mIoU = None
    best_mIoU = 0.0
    image_with_worst_mIoU = None  # --- IGNORE ---
    worst_mIoU = 1.0

    for i in range(num_images):
        # Affichage de l'image initiale, du masque prédit et du masque de vérité terrain si disponible
        image_utils.show_image_and_masks(
            original_image_paths[i],
            ground_truth_masks_dir,
            CLASS_MAPPING,
            PALETTE,
            predicted_masks_dir, None
        )

        # Evaluation simple si masque de vérité terrain disponible
        ground_truth_mask_path = image_utils.search_for_ground_truth_mask_path(original_image_paths[i], ground_truth_masks_dir)
        if ground_truth_mask_path is not None:
            result = image_utils.eval_segmentation_simple(
                ground_truth_mask_path,
                os.path.join(predicted_masks_dir, os.path.basename(original_image_paths[i]).replace('image', 'mask')),
                CLASS_MAPPING,
                PALETTE
            )
            if result["accuracy"] > best_accuracy:
                best_accuracy = result["accuracy"]
                image_with_best_accuracy = original_image_paths[i]
            if result["accuracy"] < worst_accuracy:  # --- IGNORE ---
                worst_accuracy = result["accuracy"]
                image_with_worst_accuracy = original_image_paths[i]
            if result["mIoU"] > best_mIoU:
                best_mIoU = result["mIoU"]
                image_with_best_mIoU = original_image_paths[i]
            if result["mIoU"] < worst_mIoU:  # --- IGNORE ---
                worst_mIoU = result["mIoU"]
                image_with_worst_mIoU = original_image_paths[i]
        else:
            print("Aucun masque de vérité terrain trouvé pour cette image.")
    if image_with_best_accuracy:
        print(f"\nL'image avec la meilleure précision est : {image_with_best_accuracy} avec une précision de {best_accuracy:.4f}")
    if image_with_best_mIoU:
        print(f"\nL'image avec le meilleur mIoU est : {image_with_best_mIoU} avec un mIoU de {best_mIoU:.4f}")
    if image_with_worst_accuracy:  # --- IGNORE ---
        print(f"\nL'image avec la pire précision est : {image_with_worst_accuracy} avec une précision de {worst_accuracy:.4f}")
    if image_with_worst_mIoU:  # --- IGNORE ---
        print(f"\nL'image avec le pire mIoU est : {image_with_worst_mIoU} avec un mIoU de {worst_mIoU:.4f}")

# Afficher les résultats du batch
print("\nAffichage des résultats du batch...", image_paths)
display_results(image_paths)

## Conclusion et Prochaines Étapes

Félicitations ! Vous avez appris à :
- Configurer les appels à l'API d'inférence Hugging Face.
- Envoyer des images pour la segmentation.
- Interpréter les résultats (avec l'aide des fonctions fournies).
- Visualiser les segmentations.

Pistes d'amélioration ou d'exploration :
- **Gestion d'erreurs plus fine** : Implémenter des tentatives multiples (retry) en cas d'échec de l'API (par exemple, si le modèle est en cours de chargement).
- **Appels asynchrones** : Pour un grand nombre d'images, des appels asynchrones (avec `asyncio` et `aiohttp`) seraient beaucoup plus rapides.
- **Autres modèles** : Explorer d'autres modèles de segmentation ou d'autres tâches sur Hugging Face Hub.

N'hésitez pas à modifier le code, à tester avec vos propres images et à explorer davantage !

**_Note_** : Si vous aimez ce modèle, n'hésitez pas à le [télécharger](https://huggingface.co/sayeed99/segformer_b3_clothes) et jouer avec directement sur votre machine !

## Annexe 1. Evaluation de deux masques

Nous allons maintenant évaluer la qualité d'un masque de segmentation prédite par rapport à un masque de vérité terrain, en utilisant des métriques telles que l'Intersection over Union (IoU) et la précision.


In [ ]:
# TEST : Evaluation de deux masques
# Recherche du masque de vérité terrain pour une image donnée
ground_truth_mask_path = image_utils.search_for_ground_truth_mask_path(
    "../assets/ground_truth_masks/image_24.png", ground_truth_masks_dir
)
if ground_truth_mask_path:
	print(image_utils.eval_segmentation_simple(ground_truth_mask_path, "../assets/predicted_masks/mask_24.png", CLASS_MAPPING, PALETTE))
else :
	print("Aucun masque de vérité terrain trouvé pour cette image.")

### Annexe 2 : 🕞 Comparaison entre des requêtes synchrones et asynchrones (request vs aiohttp)

In [ ]:
# Essai / comparaison entre request et asyncio / aiohttp

# ------------------ VERSION SYNCHRONE ------------------
def segment_images_sync(image_paths, headers=headers, timeout=30):
    results = []
    for img_path in image_paths:
        with open(img_path, "rb") as f:
            image_data = f.read()
        ext = os.path.splitext(img_path)[1].lower()
        headers["Content-Type"] = f"image/{ext[1:]}"
        try:
            resp = requests.post(
                API_URL, headers=headers, data=image_data, timeout=timeout
            )
            results.append((img_path, resp.json() if resp.status_code == 200 else None))
        except requests.exceptions.RequestException as e:
            print(f"🌐 Erreur réseau pour {img_path} : {e}")
            results.append((img_path, None))
    return results


# ------------------ VERSION ASYNCHRONE ------------------
async def segment_images_async(image_paths, headers=headers):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for img_path in image_paths:
            with open(img_path, "rb") as f:
                image_data = f.read()
            ext = os.path.splitext(img_path)[1].lower()
            headers["Content-Type"] = f"image/{ext[1:]}"
            tasks.append(session.post(API_URL, headers=headers, data=image_data))
        responses = await asyncio.gather(*tasks)
        results = []
        for img_path, resp in zip(image_paths, responses):
            results.append((img_path, await resp.json()))
        return results


# ------------------ BENCHMARK ------------------
print("⏱️ Benchmark SYNCHRONE")
start = time.time()
results_sync = segment_images_sync(image_paths, headers=headers)
sync_time = time.time() - start
print(f"Temps total synchrone : {sync_time:.2f}s\n")

print("⏱️ Benchmark ASYNCHRONE")
start = time.time()
results_async = await segment_images_async(image_paths, headers=headers)
async_time = time.time() - start
print(f"Temps total asynchrone : {async_time:.2f}s\n")



#### Annexe 3 : ⚠️ En cas de non disponibilité d'Hugging Face API, vous pouvez utiliser le modèle en local avec la bibliothèque `transformers` de Hugging Face.




In [ ]:
# Test du modèle en local

%matplotlib inline

# 1. Charger modèle + processeur (une seule fois, mis en cache local ensuite)
model_name = "sayeed99/segformer_b3_clothes"
model = AutoModelForSemanticSegmentation.from_pretrained(model_name)
processor = AutoImageProcessor.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Liste des chemins d’images
print(f"Nombre d'images trouvées : {len(image_paths)}")

# Boucle sur toutes les images
start_time = time.time()
for single_image_path in image_paths:
    print(f"Traitement de l'image : {single_image_path}")

    try:
        image = Image.open(single_image_path)
    except OSError as e:
        print(f"📂 Erreur lecture fichier : {e}")
        continue  # passe à l’image suivante

    # Prétraitement
    inputs = processor(images=image, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Inférence
    with torch.no_grad():
        outputs = model(**inputs)

    # Post-traitement
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(
        logits, size=image.size[::-1], mode="bilinear", align_corners=False  # (H, W)
    )
    pred_seg = upsampled_logits.argmax(dim=1)[0].cpu().numpy().astype(np.uint8)

    image_utils.show_image_and_masks(
        single_image_path, ground_truth_masks_dir, CLASS_MAPPING, PALETTE, None, pred_seg
    )
    ground_truth_mask_path = image_utils.search_for_ground_truth_mask_path(single_image_path, ground_truth_masks_dir)
    if ground_truth_mask_path is not None:
        image_utils.eval_segmentation_simple(
            ground_truth_mask_path,
            os.path.join(
				predicted_masks_dir,
				os.path.basename(single_image_path).replace("image", "mask"),
			),
            CLASS_MAPPING,
            PALETTE
        )
    else:
        print("Aucun masque de vérité terrain trouvé pour cette image.")

    # Sauvegarde masque
    mask_save_path = os.path.join(
        predicted_masks_dir,
        os.path.basename(single_image_path).replace("image", "mask"),
    )
    Image.fromarray(pred_seg).save(mask_save_path)
    print(f"🖼️ Masque sauvegardé : {mask_save_path}")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"⏱️ Temps total pour {len(image_paths)} images : {elapsed_time:.2f} secondes")
print(f"⏱️ Temps moyen par image : {elapsed_time / len(image_paths):.2f} secondes")